In [1]:
import numpy as np
import pandas as pd

In [4]:
df=pd.read_excel("Data set -Churn modelling.xlsx", sheetname="Data")

C:\Users\sreekar chidurala\Anaconda3\envs\sreekar\lib\site-packages\pandas\io\excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


In [5]:

df.head()

,Churn,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,0,128,1,1,2.7,1,265.1,110,89.0,9.87,10.0
1,0,107,1,1,3.7,1,161.6,123,82.0,9.78,13.7
2,0,137,1,0,0.0,0,243.4,114,52.0,6.06,12.2
3,0,84,0,0,0.0,2,299.4,71,57.0,3.10,6.6
4,0,75,0,0,0.0,3,166.7,113,41.0,7.42,10.1


In [6]:
df.isnull().sum()

Churn              0
AccountWeeks       0
ContractRenewal    0
DataPlan           0
DataUsage          0
CustServCalls      0
DayMins            0
DayCalls           0
MonthlyCharge      0
OverageFee         0
RoamMins           0
dtype: int64

In [7]:
X=df.drop(labels="Churn",axis=1)
y=df["Churn"]

In [8]:
import statsmodels.api as sm
from patsy import dmatrices
def stepwise_selection(X,y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.argmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))
                
        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

result = stepwise_selection(X, y)

print('resulting features:')
print(result)


C:\Users\sreekar chidurala\Anaconda3\envs\sreekar\lib\site-packages\ipykernel_launcher.py:33: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.


Add  ContractRenewal                with p-value 1.43361e-52
Add  CustServCalls                  with p-value 9.11945e-39
Add  DayMins                        with p-value 1.12768e-33
Add  DataPlan                       with p-value 4.09478e-10
Add  OverageFee                     with p-value 7.43216e-09
Add  RoamMins                       with p-value 9.48416e-05
resulting features:
['ContractRenewal', 'CustServCalls', 'DayMins', 'DataPlan', 'OverageFee', 'RoamMins']


In [28]:
new=df[["ContractRenewal","CustServCalls","DayMins","DataPlan","OverageFee","RoamMins" ]]

In [29]:
new.shape

(3333, 6)

In [30]:
#using stepwise features
from sklearn.model_selection import train_test_split
train_x, test_x, y_train, y_test = train_test_split(new,y,test_size=0.3, random_state=0)

In [34]:
train_x.shape

(2333, 6)

In [31]:
## scalling

In [42]:
from sklearn.preprocessing import MaxAbsScaler
scaler = MaxAbsScaler()
scaler.fit(train_x)
X_train1 = scaler.transform(train_x)
X_test1 = scaler.transform(test_x)

In [43]:
X_train1.shape

(2333, 10)

In [45]:
#imports
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, make_scorer
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train1, y_train)
y_predict = model.predict(X_test1)
model_score = model.score(X_test1, y_test)
print(model_score)
print(metrics.confusion_matrix(y_test, y_predict))

0.869
[[849  13]
 [118  20]]


C:\Users\sreekar chidurala\Anaconda3\envs\sreekar\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [46]:
print(classification_report(y_test,y_predict))

              precision    recall  f1-score   support

           0       0.88      0.98      0.93       862
           1       0.61      0.14      0.23       138

   micro avg       0.87      0.87      0.87      1000
   macro avg       0.74      0.56      0.58      1000
weighted avg       0.84      0.87      0.83      1000



In [47]:
## using all features

In [48]:
from sklearn.model_selection import train_test_split
train_x, test_x, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)

In [49]:
from sklearn.preprocessing import MaxAbsScaler
scaler = MaxAbsScaler()
scaler.fit(train_x)
X_train = scaler.transform(train_x)
X_test = scaler.transform(test_x)

In [50]:
X_train.shape

(2333, 10)

In [51]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_predict = model.predict(X_test)
model_score = model.score(X_test, y_test)
print(model_score)
print(metrics.confusion_matrix(y_test, y_predict))

0.869
[[849  13]
 [118  20]]


C:\Users\sreekar chidurala\Anaconda3\envs\sreekar\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
